# Train models on satellite image dataset

In [1]:
import os
import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data.dataset as Dataset

from models import *

## Load dataset

load images data

In [2]:
# get image file list from a directory
def get_img_list(dir, firelist, ext=None):
    newdir = dir
    if os.path.isfile(dir):
        if ext is None:
            firelist.append(dir)
        elif ext in dir[-3:]:
            firelist.append(dir)
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            newdir = os.path.join(dir, s)
            get_img_list(newdir, firelist, ext)
    return firelist

industry=[]
agriculture=[]
residence=[]
get_img_list("dataset/industry",industry)
get_img_list("dataset/agriculture",agriculture)
get_img_list("dataset/residence",residence)

X = []
for i in industry:
    img = cv2.imread(i)
    img =cv2.resize(img,(32,32))
    X.append(img)
    
for i in agriculture:
    img = cv2.imread(i)
    img =cv2.resize(img,(32,32))
    oneD = img.reshape(img.size,1)
    X.append(img)
    
for i in residence:
    img = cv2.imread(i)
    img =cv2.resize(img,(32,32))
    oneD = img.reshape(img.size,1)
    X.append(img)

X = np.array(X)
X = np.transpose(X, (0,3,1,2))
print(X.shape)

(370, 3, 32, 32)


build labels
- industry: 0
- agriculture: 1
- residence: 2

In [3]:
Y=[]
for i in range(len(industry)):
    Y.append(0)
for i in range(len(agriculture)):
    Y.append(1)
for i in range(len(residence)):
    Y.append(2)
print(len(Y))

370


In [4]:
# define dataset class
class SatelliteDataset(Dataset.Dataset):
    def __init__(self, Data, Label):
        self.Data = Data
        self.Label = torch.LongTensor(Label)
    def __len__(self):
        return len(self.Data)
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = self.Label[index]
        return data, label

# build dataset and dataloader
Data = X
Label = Y
trainset = SatelliteDataset(Data, Label)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=0)

## Build model

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# select a model
net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
# net = SimpleDLA()
# net = VGG('VGG19')

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

## Loss function, optimizer adn scheduler

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

## Train model

In [12]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/len(trainloader), 100.*correct/total, correct, total))

    scheduler.step()

for epoch in range(10):
    train(epoch)



Epoch: 0
Loss: 2.646 | Acc: 52.162% (193/370)

Epoch: 1
Loss: 3.640 | Acc: 58.919% (218/370)

Epoch: 2
Loss: 0.789 | Acc: 63.243% (234/370)

Epoch: 3
Loss: 2.064 | Acc: 68.919% (255/370)

Epoch: 4
Loss: 1.850 | Acc: 72.973% (270/370)

Epoch: 5
Loss: 0.848 | Acc: 72.432% (268/370)

Epoch: 6
Loss: 0.760 | Acc: 75.135% (278/370)

Epoch: 7
Loss: 1.162 | Acc: 78.649% (291/370)

Epoch: 8
Loss: 0.923 | Acc: 79.189% (293/370)

Epoch: 9
Loss: 0.604 | Acc: 83.243% (308/370)
